# StackOverflow

## Описание данных

Мы будем работать с базой данных StackOverflow — сервиса вопросов и ответов о программировании. StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам.
В базе хранятся данные о постах за 2008 год, но в таблицах есть и о более поздние оценки, которые эти посты получили. 

Таблица stackoverflow.badges
Хранит информацию о значках, которые присуждаются за разные достижения. Например, пользователь, правильно ответивший на большое количество вопросов про PostgreSQL, может получить значок postgresql. 
Поле	Описание
id	Идентификатор значка, первичный ключ таблицы
name	Название значка
user_id	Идентификатор пользователя, которому присвоили значок, внешний ключ, отсылающий к таблице users
creation_date	Дата присвоения значка
Таблица stackoverflow.post_types
Содержит информацию о типе постов. Их может быть два:
Question — пост с вопросом;
Answer — пост с ответом.
Поле	Описание
id	Идентификатор поста, первичный ключ таблицы
type	Тип поста
Таблица stackoverflow.posts
Содержит информацию о постах.
Поле	Описание
id	Идентификатор поста, первичный ключ таблицы
title	Заголовок поста
creation_date	Дата создания поста
favorites_count	Число, которое показывает, сколько раз пост добавили в «Закладки»
last_activity_date	Дата последнего действия в посте, например комментария
last_edit_date	Дата последнего изменения поста
user_id	Идентификатор пользователя, который создал пост, внешний ключ к таблице users
parent_id	Если пост написали в ответ на другую публикацию, в это поле попадёт идентификатор поста с вопросом
post_type_id	Идентификатор типа поста, внешний ключ к таблице post_types
score	Количество очков, которое набрал пост
views_count	Количество просмотров
Таблица stackoverflow.users
Содержит информацию о пользователях.
Поле	Описание
id	Идентификатор пользователя, первичный ключ таблицы
creation_date	Дата регистрации пользователя
display_name	Имя пользователя
last_access_date	Дата последнего входа
location	Местоположение
reputation	Очки репутации, которые получают за хорошие вопросы и полезные ответы
views	Число просмотров профиля пользователя
Таблица stackoverflow.vote_types
Содержит информацию о типах голосов. Голос — это метка, которую пользователи ставят посту. Типов бывает несколько: 
UpMod — такую отметку получают посты с вопросами или ответами, которые пользователи посчитали уместными и полезными.
DownMod — такую отметку получают посты, которые показались пользователям наименее полезными.
Close — такую метку ставят опытные пользователи сервиса, если заданный вопрос нужно доработать или он вообще не подходит для платформы.
Offensive — такую метку могут поставить, если пользователь ответил на вопрос в грубой и оскорбительной манере, например, указав на неопытность автора поста.
Spam — такую метку ставят в случае, если пост пользователя выглядит откровенной рекламой.
Поле	Описание
id	Идентификатор типа голоса, первичный ключ
name	Название метки
Таблица stackoverflow.votes
Содержит информацию о голосах за посты. 
Поле	Описание
id	Идентификатор голоса, первичный ключ
post_id	Идентификатор поста, внешний ключ к таблице posts
user_id	Идентификатор пользователя, который поставил посту голос, внешний ключ к таблице users
bounty_amount	Сумма вознаграждения, которое назначают, чтобы привлечь внимание к посту
vote_type_id	Идентификатор типа голоса, внешний ключ к таблице vote_types
creation_date	Дата назначения голоса

### Часть 1

1.
Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT count(p.id)
FROM stackoverflow.posts as p
join stackoverflow.post_types as pt
on p.post_type_id=pt.id
where pt.type = 'Question' and score > 300 or favorites_count>=100

2.
Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
with avgs as(SELECT count(p.id) as q_count
FROM stackoverflow.posts as p
join stackoverflow.post_types as pt
on p.post_type_id=pt.id
where pt.type = 'Question' and p.creation_date::date between '2008-11-01' and '2008-11-18'
group by p.creation_date::date)
select round (avg(q_count))
from avgs

3.
Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT COUNT(DISTINCT u.id)
from stackoverflow.users as u
inner join stackoverflow.badges as b on b.user_id=u.id
where u.creation_date::date = b.creation_date::date

4.
Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
select count(distinct p.id)
from stackoverflow.posts as p
join stackoverflow.votes as v on p.id= v.post_id
where p.user_id = (select id
FROM stackoverflow.users AS u
where display_name='Joel Coehoorn')

5. Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
select *, ROW_NUMBER() OVER (ORDER BY v.id DESC) as rank
from stackoverflow.vote_types as v
order by v.id

6.
Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
select v.user_id, count(v.id) as vote_count
from stackoverflow.votes as v 
where v.vote_type_id=(select id 
from stackoverflow.vote_types as vt
where vt.name='Close')
group by user_id
order by vote_count desc, user_id desc
limit 10

7.
Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
идентификатор пользователя;
число значков;
место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
SELECT user_id, COUNT(b.id) AS badge_c, DENSE_RANK() OVER (ORDER BY COUNT(b.id)) as rank
FROM stackoverflow.badges AS b
WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY user_id
ORDER BY badge_c DESC, user_id ASC
LIMIT 10;

8.
Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
заголовок поста;
идентификатор пользователя;
число очков поста;
среднее число очков пользователя за пост, округлённое до целого числа.
Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
select title, user_id, score, round(avg(score) over (partition by user_id))
from stackoverflow.posts
where title is not null and score!=0

9.
Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
select title
from stackoverflow.posts as p
where p.user_id = (select u.id as user_id
from stackoverflow.users as u 
join stackoverflow.badges as b on b.user_id= u.id 
group by u.id
having count(b.id) >1000)
and not p.title is null

10.
Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
пользователям с числом просмотров меньше 100 — группу 3.
Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с количеством просмотров меньше либо равным нулю не должны войти в итоговую таблицу.

In [ ]:
select u.id, 
    u.views, 
    CASE
           WHEN u.views < 100 THEN 3
           WHEN u.views >= 350 THEN 1
           ELSE 2
       END
from stackoverflow.users as u
WHERE u.location LIKE '%Canada%' and  u.views!=0

11.
Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
with rating as (select u.id, 
    u.views, 
    CASE 
           WHEN u.views < 100 THEN 3
           WHEN u.views >= 350 THEN 1
           ELSE 2
       END as v_group
from stackoverflow.users as u
WHERE u.location LIKE '%Canada%' and  u.views!=0), 
rating_best as (
select *, max(views) over (partition by v_group) as best
from rating)
select id,v_group,views
from rating_best
where views = best
order by views desc, id asc

In [ ]:
12.
Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
номер дня;
число пользователей, зарегистрированных в этот день;
сумму пользователей с накоплением.

In [ ]:
select EXTRACT(day from creation_date::date) as reg_date, count(users.id)::int, sum (count(users.id)) over(order by creation_date::date)
from stackoverflow.users
where creation_date::date between '2008-11-01' and '2008-11-30'
group by creation_date::date
order by reg_date asc

In [ ]:
13.
Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
идентификатор пользователя;
разницу во времени между регистрацией и первым постом.

In [ ]:
select distinct u.id as user_id, MIN(p.creation_date) OVER (PARTITION BY u.id)- u.creation_date
from stackoverflow.users as u
inner join stackoverflow.posts as p on p.user_id=u.id
order by user_id asc

### Часть 2

1.
Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
select DATE_TRUNC('month',creation_date::date)::date as month, sum(views_count) as m_views
from stackoverflow.posts as p
where extract (year from creation_date::date)=2008
group by month
order by m_views desc

2.
Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
SELECT u.display_name,
       COUNT(DISTINCT p.user_id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u ON p.user_id = u.id
JOIN stackoverflow.post_types AS pt ON pt.id = p.post_type_id
WHERE p.creation_date::date BETWEEN u.creation_date::date AND (u.creation_date::date + INTERVAL '1 month')
   AND pt.id =2
GROUP BY u.display_name
HAVING COUNT(p.id) > 100
ORDER BY u.display_name;

In [ ]:
3.
Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
select date_trunc('month',creation_date::date)::date, count(p.id)
from stackoverflow.posts as p
where p.user_id in (select u.id
from stackoverflow.posts as p 
join stackoverflow.users as u on u.id= p.user_id
where u.creation_date::date between '2008-09-01' and '2008-09-30'
and p.creation_date::date between '2008-12-01' and '2008-12-31')
and p.creation_date::date between '2008-01-01' and '2008-12-31'
group by date_trunc('month',creation_date::date)::date
order by date_trunc('month',creation_date::date)::date desc


4.
Используя данные о постах, выведите несколько полей:
идентификатор пользователя, который написал пост;
дата создания поста;
количество просмотров у текущего поста;
сумма просмотров постов автора с накоплением.
Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
select user_id, creation_date, views_count, sum(views_count) over (partition by user_id order by creation_date)
from stackoverflow.posts
order by user_id asc, creation_date

5.
Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
with activity as (select u.id as user_id, count (distinct p.creation_date::date)
from stackoverflow.users as u 
join stackoverflow.posts as p on u.id= p.user_id
where p.creation_date::date between '2008-12-01' and '2008-12-07'
group by u.id
order by u.id)
select round(avg(count)) from activity

6.
На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
Номер месяца.
Количество постов за месяц.
Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.
Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [ ]:
select 
    extract (month from p.creation_date::date) as month, 
    count(p.id) as post_count, 
    round((count(p.id)::numeric / LAG(count(p.id)) OVER(ORDER BY extract (month from p.creation_date::date))-1)*100,2) AS previous_m
from stackoverflow.posts as p 
where p.creation_date::date between '2008-09-01' and '2008-12-31'
group by extract (month from p.creation_date::date)

In [ ]:
7.
Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведите данные его активности за октябрь 2008 года в таком виде:
номер недели;
дата и время последнего поста, опубликованного на этой неделе.

In [ ]:

select distinct extract(week from creation_date::date), max(creation_date) over (order by extract(week from creation_date::date))
from stackoverflow.posts as p
where p.user_id = (select pp.user_id
    from stackoverflow.posts as pp
    group by pp.user_id
    order by count(pp.id) desc
    limit 1)
and p.creation_date::date between '2008-10-01' and '2008-10-31'